# Scaling Analysis Examples TTbar70

Authors: Brain Gravelle and Boyana Norris


All this is using the taucmdr python libraries from paratools
http://taucommander.paratools.com/

<a href='#top'>top</a><br>
<a href='#data'>Loading Data</a><br>
<a href='#metrics'>Metrics</a><br>
<a href='#corr'>Correlations</a><br>
<a href='#stl_sb'>Store Buffer Stalls</a><br>
<a href='#fetch_stalls'>Fetch Stalls</a><br>
<a href='#l3'>l3 misses</a><br>

## Imports
This section imports necessary libraies, the metrics.py and utilities.py files and sets up the window.


<a id='top'></a>

In [6]:
# A couple of scripts to set the environent and import data from a .tau set of results
from utilities import *
from metrics import *
# Plotting, notebook settings:
%matplotlib inline  
#plt.rcParams.update({'font.size': 16})
import numbers
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.float_format', lambda x: '%.2e' % x)
pd.set_option('display.max_columns',100)
pd.set_option('max_colwidth', 70)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import copy, shelve

In [7]:
application = "pennant"
exp = "example"

data = load_perf_data(application,exp,nolibs=False,scaling=False,callpaths=True,data_dir="tau")

print data.keys()

['PAPI_L2_TCA', 'PAPI_VEC_DP', 'PAPI_L2_TCM', 'PAPI_VEC_SP', 'PAPI_LST_INS', 'PAPI_L1_TCM', 'PAPI_TOT_CYC', 'METADATA']


In [9]:
data_sorted = data['PAPI_TOT_CYC'][['Exclusive']].sort_values(by='Exclusive',ascending=False)
data_sorted.head(5)

Exclusive
rank context thread region                                                                             
0    0       60     [SUMMARY] bool _INTERNAL_25_______src_kmp_barrier_cpp_38a91946::__k...  12070645045
                    [SUMMARY] .TAU application  => [CONTEXT] .TAU application  => [SAMP...  12070645045
             62     [SUMMARY] .TAU application  => [CONTEXT] .TAU application  => [SAMP...  11394373758
                    [SUMMARY] bool _INTERNAL_25_______src_kmp_barrier_cpp_38a91946::__k...  11394373758
             54     [SUMMARY] bool _INTERNAL_25_______src_kmp_barrier_cpp_38a91946::__k...  11283502633

In [14]:
alldata = combine_metrics(data,inc_exc='Exclusive')

cm = sns.light_palette("yellow", as_cmap=True)
metric_li = ['PAPI_L2_TCA', 'PAPI_VEC_DP', 'PAPI_L2_TCM', 'PAPI_VEC_SP', 'PAPI_LST_INS', 'PAPI_L1_TCM', 'PAPI_TOT_CYC']
corr_data = copy.deepcopy(alldata)
corr_data = corr_data[corr_data.index.get_level_values('region').str.contains('FindCandidatesClone')]
# corr_data.head(10)
correlations_pearson = alldata.corr('pearson').fillna(0)    # Other methods: 'kendall', 'spearman'
correlations_pearson.style.format("{:.2%}").background_gradient(cmap=cm)

,PAPI_TOT_CYC,PAPI_L2_TCA,PAPI_VEC_DP,PAPI_L2_TCM,PAPI_VEC_SP,PAPI_LST_INS,PAPI_L1_TCM
PAPI_TOT_CYC,100.00%,96.28%,84.88%,89.75%,0.00%,86.20%,95.52%
PAPI_L2_TCA,96.28%,100.00%,86.71%,91.00%,0.00%,87.42%,96.04%
PAPI_VEC_DP,84.88%,86.71%,100.00%,94.65%,0.00%,94.41%,85.01%
PAPI_L2_TCM,89.75%,91.00%,94.65%,100.00%,0.00%,94.50%,89.93%
PAPI_VEC_SP,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
PAPI_LST_INS,86.20%,87.42%,94.41%,94.50%,0.00%,100.00%,85.95%
PAPI_L1_TCM,95.52%,96.04%,85.01%,89.93%,0.00%,85.95%,100.00%
